# Explore here

In [1]:
# Your code here
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import numpy as np
from numpy.random import seed, choice
from sklearn.tree import DecisionTreeClassifier
from collections import Counter
from sklearn.metrics import *
from imblearn.metrics import specificity_score
import pandas as pd

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/decision-tree-project-tutorial/main/diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
X = df.drop('Outcome', axis=1)
y = df['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
def get_metrics(y_train, y_test, y_pred_train, y_pred_test):
    # Calcular métricas para el conjunto de entrenamiento
    train_accuracy = accuracy_score(y_train, y_pred_train)
    train_f1 = f1_score(y_train, y_pred_train)
    train_auc = roc_auc_score(y_train, y_pred_train)
    train_precision = precision_score(y_train, y_pred_train)
    train_recall = recall_score(y_train, y_pred_train)
    train_specificity = specificity_score(y_train, y_pred_train)

    # Calcular métricas para el conjunto de prueba
    test_accuracy = accuracy_score(y_test, y_pred_test)
    test_f1 = f1_score(y_test, y_pred_test)
    test_auc = roc_auc_score(y_test, y_pred_test)
    test_precision = precision_score(y_test, y_pred_test)
    test_recall = recall_score(y_test, y_pred_test)
    test_specificity = specificity_score(y_test, y_pred_test)

    # Calcular la diferencia entre métricas de entrenamiento y prueba
    diff_accuracy = train_accuracy - test_accuracy
    diff_f1 = train_f1 - test_f1
    diff_auc = train_auc - test_auc
    diff_precision = train_precision - test_precision
    diff_recall = train_recall - test_recall
    diff_specificity = train_specificity - test_specificity

    # Crear un DataFrame con los resultados
    metrics_df = pd.DataFrame([[train_accuracy, train_f1, train_auc, train_precision, train_recall, train_specificity],[test_accuracy, test_f1, test_auc, test_precision, test_recall, test_specificity],[diff_accuracy, diff_f1, diff_auc, diff_precision, diff_recall, diff_specificity]],
                              columns = ['Accuracy', 'F1', 'AUC', 'Precision', 'Recall', 'Specificity'],
                              index = ['Train','Test', 'Diferencia'])

    return metrics_df

In [5]:
# Cargamos el conjunto de datos de ejemplo (breast cancer)
data = load_breast_cancer()
X = data.data
y = data.target

# Dividimos los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
np.ones(10)/10

array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])

In [7]:
class CustomBoosting:

  def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=10, min_samples_leaf=10, max_features=1.0, random_state=42):
        self.n_estimators = n_estimators  # Número de estimadores en el ensamble
        self.learning_rate = learning_rate  # Tasa de aprendizaje
        self.max_depth = max_depth  # Profundidad máxima de los árboles de decisión
        self.min_samples_leaf = min_samples_leaf  # Número mínimo de muestras por hoja
        self.max_features = max_features  # Proporción máxima de características a considerar en cada división
        self.random_state = random_state  # Semilla aleatoria para reproducibilidad
        self.estimadores = [] # Almacena los estimadores del ensamble

  def muestreo(self, X, y, pesos):
    # Semilla para replicar el experimento
    np.random.seed(self.random_state)
    # Selecciono lo indices que serán usados para generar la muestra
    indices = np.random.choice(a=len(X), size=len(X), p=pesos)
    return X[indices], y[indices]

  def get_error(self, y, y_pred, pesos):
    # Obtener las instancias mal clasificadas
    instancias_erroneas = (y!=y_pred)
    # Obtenemos los pesos de las instancias erróneamente clasificadas
    pesos_instancias_erroneas = pesos[instancias_erroneas]
    # Suma total del peso de las instancias erróneas
    suma_pesos_error = np.sum(pesos_instancias_erroneas)
    # Normalizamos el error para que tome un valor entre 0 y 1
    error_ponderado = suma_pesos_error / np.sum(pesos)
    return instancias_erroneas, error_ponderado

  def get_estimator_importance(self, error_ponderado):
    # Proporción de acierto frente al error
    return np.log((1-error_ponderado)/error_ponderado)*self.learning_rate

  def update_pesos(self, estimator_importance, instancias_erroneas, pesos):
    # Actualiza los pesos de las instancias para dar más importancia a las instancias mal clasificadas
    # incrementar de forma más específica los pesos de las instancias que fueron mal clasificadas,
    # mientras que disminuye los de las que fueron correctamente clasificadas, ajustando así de manera más efectiva la atención del modelo hacia las instancias que más necesita aprender.
    actualizacion_pesos = np.where(instancias_erroneas, np.exp(estimator_importance), np.exp(-estimator_importance))
    pesos = pesos * actualizacion_pesos  # Aplica la actualización a los pesos
    pesos = pesos / np.sum(pesos)  # Normaliza los pesos para que sumen 1, creando una distribución de probabilidad válida
    return pesos



  def fit(self, X, y):
    # Inicializar los pesos de las instancias
    pesos = np.ones(len(X))/len(X)
    # Realizamos el entrenamiento de los estimadores
    for _ in range(self.n_estimators):
      # Realizamos un muestreo del dataset teniendo en cuenta el peso asociado a cada instancia
      X_sampled, y_sampled = self.muestreo(X, y, pesos)
      # Entrenamos el estimador con la versión muestreada
      estimador = DecisionTreeClassifier(max_depth=self.max_depth, min_samples_leaf=self.min_samples_leaf, max_features=self.max_features).fit(X_sampled, y_sampled)
      # Realizamos predicciones con el estimador entrenado
      y_pred = estimador.predict(X)
      # Obtener el error cometido por el estimador
      instancias_erroneas, error_ponderado = self.get_error(y, y_pred, pesos)
      # Calcular la importancia del estimador
      estimator_importance = self.get_estimator_importance(error_ponderado)
      # Almacenar el estimador junto a su importancia
      self.estimadores.append((estimador, estimator_importance))
      # Actualizar los pesos de las instancias
      pesos = self.update_pesos(estimator_importance, instancias_erroneas, pesos)

  def predict(self, X):
    # Inicializamos una variable donde almacenar las predicciones finales
    final_predictions = np.zeros(len(X))
    # Para cada estimador del ensamblado
    for estimador, estimador_imp in self.estimadores:
      # Aplicar el estimador sobre los datos a predecir
      predicciones = estimador.predict(X)
      # Pondero las precciones
      preds_ponderada = predicciones*estimador_imp
      # Acumula las predicciones ponderadas
      final_predictions = final_predictions+preds_ponderada
    mean_val = np.mean(final_predictions)
    yhats = np.where(final_predictions<mean_val,0,1)
    return yhats

In [8]:
# Entrenamos el modelo de Boosting
clf = CustomBoosting(n_estimators=10, learning_rate=0.01, max_depth=7, min_samples_leaf=7, max_features=X_train.shape[1]//2)
clf.fit(X_train, y_train)

# Realizamos predicciones en el conjunto de entrenamiento y prueba
test_pred = clf.predict(X_test)
train_pred = clf.predict(X_train)

# Evaluamos el rendimiento del modelo
get_metrics(y_train, y_test, train_pred, test_pred)

,Accuracy,F1,AUC,Precision,Recall,Specificity
Train,0.967033,0.973730,0.965304,0.975439,0.972028,0.958580
Test,0.947368,0.957746,0.943990,0.957746,0.957746,0.930233
Diferencia,0.019665,0.015984,0.021314,0.017692,0.014281,0.028347
